<a href="https://colab.research.google.com/github/jamesemansfield2/Finance_loan_approval_genai_llm/blob/main/Loan_Approval_LLM_Workflow_Colab_Drive_Live_CompanyCell_20250915v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Loan Approval — LLM Workflow (Colab • Drive • Live Gemini • Company & Limit)

**Last updated:** 2025-09-15 06:13

Colab-first, synthetic-only pipeline:
- Map Google Drive
- Set `company_to_assess` in its own cell (Step **2a**)
- Generate both **application-level** data and a **company profile** (debt/equity, profitability, credit score, lawsuit etc.)
- Call **live Gemini 2.5 Flash** for per-app JSON and a **company-level narrative**
- Limit how many applications are processed for speed/cost
- Save to: `/content/drive/Shareddrives/1a_Finance_LoanAI/loan`



---
## 0) Map Google Drive (run this first)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



---
## 1) Install deps & set API key (placeholders)

Uncomment both lines to install and set your key.


In [2]:
%pip install -q google-generativeai pydantic>=2.5.0 pandas scikit-learn numpy jsonschema python-dotenv
%env GEMINI_API_KEY=#[enter gemini api key]

env: GEMINI_API_KEY=#[enter gemini api key]



---
## 2) Setup & Basic Configuration


In [ ]:
# ==== 2) Setup & Basic Configuration ====

import os, time, json, uuid, random
from typing import List, Optional, Literal, Dict, Any

import numpy as np
import pandas as pd

from pydantic import BaseModel, Field, field_validator
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from jsonschema import validate as json_validate, Draft202012Validator
from jsonschema.exceptions import ValidationError as JSONSchemaError
import google.generativeai as genai

# --- Basic knobs (EDIT HERE if needed) ---
N_SYNTHETIC: int = int(os.getenv("N_SYNTHETIC", "500"))

# Where results will be written (Drive is already mounted in Step 0)
OUTPUTS_DIR = "/content/drive/Shareddrives/1a_Finance_LoanAI/loan"  # <<< EDIT PATH IF YOU WANT
os.makedirs(OUTPUTS_DIR, exist_ok=True)

# LLM config (API key is set in Step 1)
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
LLM_MODEL_NAME = os.getenv("LLM_MODEL_NAME", "gemini-2.5-flash")     # <<< change model if desired

# Feature policy (allowed inputs to the LLM/rules)
APP_ID_COL = "application_id"
ALLOWED_FEATURES = [
    "annual_income", "loan_amount", "interest_rate", "term_months",
    "fico_score", "employment_years", "dti", "purpose", "state_code",
    "num_open_accounts", "delinquencies_2y", "revol_util"
]
CATEGORICALS = ["purpose", "state_code"]
NUMERICS = [f for f in ALLOWED_FEATURES if f not in CATEGORICALS]

# Rules thresholds (EDIT to match your underwriting policy)
DTI_APPROVE_MAX      = float(os.getenv("DTI_APPROVE_MAX", "0.36"))
DTI_CONDITIONAL_MAX  = float(os.getenv("DTI_CONDITIONAL_MAX", "0.48"))
FICO_APPROVE_MIN     = int(os.getenv("FICO_APPROVE_MIN", "720"))
FICO_CONDITIONAL_MIN = int(os.getenv("FICO_CONDITIONAL_MIN", "660"))

# --- LLM retry & throttle (used in Step 6/7) ---
# Increase THROTTLE_SECONDS if you see rate limits; increase retries for flakiness.
MAX_LLM_RETRIES   = int(os.getenv("MAX_LLM_RETRIES", "1"))
BACKOFF_BASE_SEC  = float(os.getenv("BACKOFF_BASE_SEC", "1.5"))
BACKOFF_JITTER    = float(os.getenv("BACKOFF_JITTER", "0.25"))
THROTTLE_SECONDS  = float(os.getenv("THROTTLE_SECONDS", "0.0"))

# --- Per-call timeout + optional fallback (Step 6/7 use these) ---
CALL_TIMEOUT_SEC   = float(os.getenv("CALL_TIMEOUT_SEC", "20"))  # hard stop per LLM call
FALLBACK_TO_SINGLE = True  # try single-item calls if a batch fails/times out

# Tracing (set False to silence per-app LLM latency lines)
TRACE_LOGS = True  # <<< toggle to False if you want quieter logs

# Reproducibility
np.random.seed(42)
random.seed(42)

print("Configured.")
print("  Output folder:", OUTPUTS_DIR)
print("  N_SYNTHETIC:", N_SYNTHETIC)
print("  LLM model:", LLM_MODEL_NAME)
if not GEMINI_API_KEY:
    print("  ⚠️  GEMINI_API_KEY not set yet; uncomment Step 1 to set it.")
print("  Note: set company_to_assess & MAX_APPS_PER_COMPANY in Step 2a.")


Configured.
  Output folder: /content/drive/Shareddrives/1a_Finance_LoanAI/loan
  N_SYNTHETIC: 500
  LLM model: gemini-2.5-flash
  Note: set company_to_assess & MAX_APPS_PER_COMPANY in Step 2a.



---
## 2a) Set Company & Limit (EDIT HERE)

Change the company key and the processing limit as needed.


In [ ]:

company_to_assess: str = "COMP123"   # <<< edit this
MAX_APPS_PER_COMPANY: int = 5       # <<< cap how many applications to process for this company

print("company_to_assess:", company_to_assess)
print("MAX_APPS_PER_COMPANY:", MAX_APPS_PER_COMPANY)


company_to_assess: COMP123
MAX_APPS_PER_COMPANY: 5



---
## 3) Schema & Validation (Pydantic + JSON Schema)


In [ ]:

class Application(BaseModel):
    application_id: str
    company_id: str
    annual_income: float
    loan_amount: float
    interest_rate: Optional[float] = None
    term_months: int
    fico_score: int
    employment_years: float
    dti: float
    purpose: str
    state_code: str
    num_open_accounts: Optional[int] = 0
    delinquencies_2y: Optional[int] = 0
    revol_util: Optional[float] = 0.0

    @field_validator("dti")
    @classmethod
    def dti_in_0_1(cls, v):
        if not (0 <= v <= 1.0):
            raise ValueError("DTI must be 0..1 fraction")
        return v

    @field_validator("fico_score")
    @classmethod
    def fico_range(cls, v):
        if not (300 <= v <= 850):
            raise ValueError("FICO must be in 300..850")
        return v

class LLMDecision(BaseModel):
    version: str = "1.0"
    decision: Literal["approve", "conditional", "deny"]
    reasons: List[str]
    missing_documents: List[str] = []
    policy_citations: List[str] = []
    risk_summary: str
    risk_score_0_1: Optional[float] = Field(default=None, ge=0.0, le=1.0)
    fair_lending_flags: List[str] = []

class FinalDecision(BaseModel):
    application_id: str
    company_id: str
    baseline_decision: Literal["approve", "conditional", "deny"]
    llm_decision: LLMDecision
    combined_decision: Literal["approve", "conditional", "deny"]
    notes: str = ""

# Company profile schema (for the narrative stage)
class CompanyProfile(BaseModel):
    company_id: str
    company_name: str
    debt_to_equity: float
    profit_margin: float
    roa: float
    roe: float
    credit_score: int
    lawsuit_flag: bool
    lawsuit_summary: Optional[str] = ""

LLM_DECISION_JSON_SCHEMA: Dict[str, Any] = LLMDecision.model_json_schema()
validator = Draft202012Validator(LLM_DECISION_JSON_SCHEMA)
print("Schemas ready.")


Schemas ready.



---
## 4) Synthetic Data + Company Profiles

Generates:
- Application-level rows with `company_id`
- A **company profile** table with fields used in the narrative (D/E, profitability, credit score, lawsuit flag/summary)

Ensures `company_to_assess` has coverage and a realistic profile. For **COMP123**, we set:
- Debt-to-Equity ≈ **0.8**
- Credit Score ≈ **750**
- `lawsuit_flag=True` with a short summary (so the LLM can reference **Chunk 1**).


In [ ]:

US_STATE_ABBRS = [
    "AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME",
    "MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA",
    "RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"
]
PURPOSES = ["debt_consolidation","home_improvement","major_purchase","medical","auto","other"]

def amortized_payment(principal: float, annual_rate: float, term_months: int) -> float:
    r = annual_rate / 12.0
    if r <= 0:
        return principal / max(term_months, 1)
    denom = (1 - (1 + r)**(-term_months))
    return principal * r / max(denom, 1e-9)

def generate_synthetic_applications(n: int = 500, seed: int = 42, include_company: str = "COMP123") -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    incomes = np.exp(rng.normal(11.1, 0.55, size=n))
    fico = np.clip(rng.normal(700, 60, size=n).round(), 580, 800).astype(int)

    term_choices = np.array([36, 60, 72])
    term_probs = np.array([0.25, 0.6, 0.15])
    terms = rng.choice(term_choices, size=n, p=term_probs)

    base_rate = 0.22 - (fico - 580) * (0.22 - 0.07) / (800 - 580)
    interest_rates = np.clip(base_rate + rng.normal(0, 0.01, size=n), 0.05, 0.28)

    loan_frac = rng.uniform(0.1, 0.6, size=n)
    loan_amount = (incomes * loan_frac).round(2)

    employment_years = np.clip(rng.poisson(6, size=n), 0, 40).astype(float)
    revol_util = np.clip(rng.normal(0.45, 0.2, size=n), 0, 0.95)
    num_open_accounts = rng.integers(2, 20, size=n)
    delinq_2y = rng.binomial(3, 0.15, size=n)

    purposes = rng.choice(PURPOSES, size=n)
    states = rng.choice(US_STATE_ABBRS, size=n)

    existing_obl_ratio = np.clip(rng.normal(0.18, 0.08, size=n), 0.05, 0.5)
    existing_obl = incomes * existing_obl_ratio / 12.0

    monthly_payment = np.array([amortized_payment(p, r, t) for p, r, t in zip(loan_amount, interest_rates, terms)])
    monthly_income = incomes / 12.0
    dti = (existing_obl + monthly_payment) / np.maximum(monthly_income, 1.0)
    dti = np.clip(dti, 0, 0.9)

    companies = [f"COMP{i:03d}" for i in range(100, 200)]
    base_companies = rng.choice(companies, size=n)
    force_k = max(20, n // 10)
    base_companies[:force_k] = include_company
    rng.shuffle(base_companies)

    df = pd.DataFrame({
        "application_id": [f"A{100000+i}" for i in range(n)],
        "company_id": base_companies,
        "annual_income": incomes.round(2),
        "loan_amount": loan_amount,
        "interest_rate": interest_rates.round(4),
        "term_months": terms,
        "fico_score": fico,
        "employment_years": employment_years,
        "dti": dti.round(4),
        "purpose": purposes,
        "state_code": states,
        "num_open_accounts": num_open_accounts,
        "delinquencies_2y": delinq_2y,
        "revol_util": revol_util.round(4)
    })
    return df

def generate_company_profiles(apps: pd.DataFrame, seed: int = 42) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    comps = sorted(apps["company_id"].unique().tolist())
    rows = []
    for cid in comps:
        # defaults
        d2e = float(np.clip(rng.normal(0.6, 0.25), 0.1, 1.5))
        pm  = float(np.clip(rng.normal(0.15, 0.08), 0.02, 0.45))
        roa = float(np.clip(rng.normal(0.08, 0.05), 0.01, 0.25))
        roe = float(np.clip(rng.normal(0.14, 0.1), 0.02, 0.45))
        cs  = int(np.clip(int(rng.normal(720, 40)), 600, 800))
        lawsuit_flag = bool(rng.random() < 0.08)
        lawsuit_summary = "No material litigation disclosed."

        # special case: ensure the focus company matches your expected example
        if cid == "COMP123":
            d2e = 0.8
            cs = 750
            lawsuit_flag = True
            lawsuit_summary = (
                "Patent infringement lawsuit filed against Example Corp; "
                "potential damages material; outcome pending; impact assessment ongoing."
            )
            company_name = "Example Corp"
        else:
            company_name = f"Company {cid[-3:]}"

        rows.append({
            "company_id": cid,
            "company_name": company_name,
            "debt_to_equity": round(d2e, 2),
            "profit_margin": round(pm, 2),
            "roa": round(roa, 2),
            "roe": round(roe, 2),
            "credit_score": cs,
            "lawsuit_flag": lawsuit_flag,
            "lawsuit_summary": lawsuit_summary if lawsuit_flag else ""
        })
    return pd.DataFrame(rows)

apps_df = generate_synthetic_applications(N_SYNTHETIC, seed=42, include_company=company_to_assess)
profiles_df = generate_company_profiles(apps_df, seed=42)

company_df = apps_df.query("company_id == @company_to_assess").copy()
company_profile = profiles_df.loc[profiles_df["company_id"] == company_to_assess].iloc[0].to_dict()

# limit number of applications
if MAX_APPS_PER_COMPANY and len(company_df) > MAX_APPS_PER_COMPANY:
    company_df = company_df.sample(n=MAX_APPS_PER_COMPANY, random_state=42).sort_values(APP_ID_COL).reset_index(drop=True)

print(f"Generated {len(apps_df):,} rows total.")
print(f"{company_to_assess} applications selected: {len(company_df):,}")
print("Company profile:", company_profile)


Generated 500 rows total.
COMP123 applications selected: 5
Company profile: {'company_id': 'COMP123', 'company_name': 'Example Corp', 'debt_to_equity': 0.8, 'profit_margin': 0.22, 'roa': 0.18, 'roe': 0.43, 'credit_score': 750, 'lawsuit_flag': True, 'lawsuit_summary': 'Patent infringement lawsuit filed against Example Corp; potential damages material; outcome pending; impact assessment ongoing.'}



---
## 5) Baseline Decision (Rules) — DTI + FICO


In [ ]:

def baseline_rules_decision(row: pd.Series) -> str:
    dti = float(row.get("dti", 1.0))
    fico = int(row.get("fico_score", 300))
    if dti <= DTI_APPROVE_MAX and fico >= FICO_APPROVE_MIN:
        return "approve"
    if dti <= DTI_CONDITIONAL_MAX and fico >= FICO_CONDITIONAL_MIN:
        return "conditional"
    return "deny"



---
## 6) LLM (Gemini 2.5 Flash) — Per-App Strict JSON Output


In [ ]:
# ==== 6) LLM (Gemini 2.5 Flash) — Single & Batched JSON with HARD timeout (process-based) ====

import multiprocessing as mp
import json, time, random
from typing import List
from jsonschema import validate as json_validate
from jsonschema.exceptions import ValidationError as JSONSchemaError

# Uses these knobs from Step 2:
#   GEMINI_API_KEY, LLM_MODEL_NAME
#   MAX_LLM_RETRIES, BACKOFF_BASE_SEC, BACKOFF_JITTER, THROTTLE_SECONDS
#   CALL_TIMEOUT_SEC
# And these schemas/types from earlier cells:
#   LLMDecision, LLM_DECISION_JSON_SCHEMA, APP_ID_COL, ALLOWED_FEATURES

LLM_SYSTEM_POLICY = """
You are an assistant that helps loan officers evaluate consumer loan applications.
Follow risk policy and fair lending guidelines:
- Do NOT use protected attributes (race, religion, gender, age, marital status, etc.) or proxies.
- Base your analysis ONLY on the fields provided.
- Be concise, factual, and cite policy sections by name (e.g., "UW-Policy-2.1 DTI Thresholds").
- Always produce VALID JSON that conforms to the provided schema.
- If information is missing, identify specific documents needed.
- If the case is marginal, prefer "conditional" with explicit conditions.
"""

def _prompt_single(row: pd.Series, company_id: str) -> str:
    safe = {k: row.get(k) for k in [APP_ID_COL, "company_id"] + ALLOWED_FEATURES if k in row.index}
    return f"""
System policy:
{LLM_SYSTEM_POLICY}

Context:
Company to assess: "{company_id}"

Task:
Given this loan application data as JSON:
{json.dumps(safe, ensure_ascii=False, indent=2)}

Produce a STRICT JSON object matching this JSON Schema (no extra keys):
{json.dumps(LLM_DECISION_JSON_SCHEMA, ensure_ascii=False)}

Return ONLY the JSON (no prose).
"""

def _prompt_batch(rows: List[pd.Series], company_id: str) -> str:
    items = [{k: r.get(k) for k in [APP_ID_COL, "company_id"] + ALLOWED_FEATURES if k in r.index} for r in rows]
    return f"""
System policy:
{LLM_SYSTEM_POLICY}

Context:
Company to assess: "{company_id}"

Task:
For each application in the array below, return a JSON array where each element is:
{{
  "application_id": string,
  "decision": "approve" | "conditional" | "deny",
  "reasons": string[],
  "missing_documents": string[],
  "policy_citations": string[],
  "risk_summary": string,
  "risk_score_0_1": number | null,
  "fair_lending_flags": string[]
}}

Applications (JSON array):
{json.dumps(items, ensure_ascii=False, indent=2)}

Return ONLY the JSON array with one element per input, same order as the inputs.
"""

def _worker_generate(prompt: str, out_q: mp.Queue):
    """
    Runs in a separate process so we can enforce a HARD timeout by terminating the process.
    Re-creates the model inside the process (objects aren’t picklable across processes).
    """
    try:
        import google.generativeai as genai
        genai.configure(api_key=GEMINI_API_KEY)
        model = genai.GenerativeModel(
            LLM_MODEL_NAME,
            generation_config={
                "temperature": 0.2,
                "top_p": 0.9,
                "response_mime_type": "application/json",
            },
        )
        resp = model.generate_content(prompt)
        text = (getattr(resp, "text", "") or "").strip()
        if text.startswith("```"):
            text = text.replace("```json", "").replace("```", "").strip()
        out_q.put({"ok": True, "text": text})
    except Exception as e:
        out_q.put({"ok": False, "err": repr(e)})

def _call_with_hard_timeout(prompt: str, timeout_sec: float) -> str:
    """
    Start a process, wait up to timeout_sec. If it runs long, terminate and raise.
    """
    ctx = mp.get_context("spawn")  # safe in Colab
    q = ctx.Queue()
    p = ctx.Process(target=_worker_generate, args=(prompt, q))
    p.start()
    p.join(timeout_sec)
    if p.is_alive():
        p.terminate()
        p.join(1)
        raise RuntimeError(f"LLM call exceeded {timeout_sec}s (hard timeout)")
    if not q.empty():
        msg = q.get()
        if msg.get("ok"):
            return msg.get("text", "")
        raise RuntimeError(f"LLM error: {msg.get('err')}")
    raise RuntimeError("LLM returned no data")

def _should_retry(exc) -> bool:
    s = str(exc).lower()
    return any(k in s for k in ["unavailable", "timeout", "timed out", "quota", "rate", "503", "502", "500", "429"])

def call_llm_and_validate(row: pd.Series, company_id: str) -> 'LLMDecision':
    if THROTTLE_SECONDS:
        time.sleep(THROTTLE_SECONDS)
    prompt = _prompt_single(row, company_id)
    attempt, last_err = 0, None
    while attempt < MAX_LLM_RETRIES:
        t0 = time.time()
        try:
            raw = _call_with_hard_timeout(prompt, CALL_TIMEOUT_SEC)
            latency = time.time() - t0
            data = json.loads(raw)
            json_validate(instance=data, schema=LLM_DECISION_JSON_SCHEMA)
            out = LLMDecision(**data)
            if TRACE_LOGS:
                print(f"[TRACE] LLM latency={latency:.2f}s id={row.get(APP_ID_COL)} decision={out.decision}", flush=True)
            return out
        except Exception as e:
            last_err = e
            attempt += 1
            if attempt < MAX_LLM_RETRIES and _should_retry(e):
                backoff = (BACKOFF_BASE_SEC ** attempt) + random.uniform(0, BACKOFF_JITTER)
                if TRACE_LOGS:
                    print(f"[TRACE] retry {attempt}/{MAX_LLM_RETRIES} in {backoff:.2f}s after error: {e}", flush=True)
                time.sleep(backoff)
                continue
            break
    raise RuntimeError(f"LLM failed after {MAX_LLM_RETRIES} attempts: {last_err}")

def call_llm_batch_and_validate(rows: List[pd.Series], company_id: str) -> dict[str, LLMDecision]:
    if THROTTLE_SECONDS:
        time.sleep(THROTTLE_SECONDS)
    prompt = _prompt_batch(rows, company_id)
    attempt, last_err = 0, None
    while attempt < MAX_LLM_RETRIES:
        t0 = time.time()
        try:
            raw = _call_with_hard_timeout(prompt, CALL_TIMEOUT_SEC)
            latency = time.time() - t0
            data = json.loads(raw)
            if not isinstance(data, list):
                raise RuntimeError("Expected JSON array from LLM.")
            results: dict[str, LLMDecision] = {}
            for obj, r in zip(data, rows):
                json_validate(instance=obj, schema=LLM_DECISION_JSON_SCHEMA)
                dec = LLMDecision(**obj)
                results[str(obj.get("application_id", r.get(APP_ID_COL)))] = dec
            if TRACE_LOGS:
                print(f"[TRACE] LLM batch latency={latency:.2f}s size={len(rows)}", flush=True)
            return results
        except Exception as e:
            last_err = e
            attempt += 1
            if attempt < MAX_LLM_RETRIES and _should_retry(e):
                backoff = (BACKOFF_BASE_SEC ** attempt) + random.uniform(0, BACKOFF_JITTER)
                if TRACE_LOGS:
                    print(f"[TRACE] batch retry {attempt}/{MAX_LLM_RETRIES} in {backoff:.2f}s after error: {e}", flush=True)
                time.sleep(backoff)
                continue
            break
    raise RuntimeError(f"LLM batch failed after {MAX_LLM_RETRIES} attempts: {last_err}")



---
## 7) Orchestrator + Batch (Limited by MAX_APPS_PER_COMPANY)


In [ ]:
# ==== 7) Orchestrator + Batch (LLM triage + cache + time budget + progress) ====

import math, time, json, os

# ---- knobs you can tweak quickly ----
BATCH_SIZE_APPS   = 4            # apps per Gemini request (try 4–10)
TIME_BUDGET_SEC   = 30          # stop after ~N seconds (saves partials)
CACHE_BASENAME    = f"llm_cache_{company_to_assess}.json"  # cache per company
CACHE_PATH        = os.path.join(OUTPUTS_DIR, CACHE_BASENAME)

def conservative_combine(baseline: str, llm_decision: LLMDecision) -> str:
    order = {"deny": 2, "conditional": 1, "approve": 0}
    return max(baseline, llm_decision.decision, key=lambda d: order[d])

def needs_llm(row: pd.Series) -> bool:
    """Only call LLM for borderline cases to cut calls."""
    dti  = float(row.get("dti", 1.0))
    fico = int(row.get("fico_score", 300))
    clear_approve = (dti <= max(0.0, DTI_APPROVE_MAX - 0.05)) and (fico >= FICO_APPROVE_MIN + 20)
    clear_deny    = (dti >= DTI_CONDITIONAL_MAX + 0.05) or (fico <= max(300, FICO_CONDITIONAL_MIN - 40))
    return not (clear_approve or clear_deny)

# ---- tiny JSON cache helpers ----
def _load_cache(path: str) -> dict:
    if os.path.exists(path):
        try:
            with open(path, "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception:
            pass
    return {}

def _save_cache(path: str, data: dict):
    tmp = path + ".tmp"
    with open(tmp, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    os.replace(tmp, path)

def run_batch_decisions(apps: pd.DataFrame, company_id: str) -> pd.DataFrame:
    if apps.empty:
        raise ValueError(f"No applications for {company_id}.")

    print(f"[INFO] preparing {len(apps)} applications for company {company_id}...", flush=True)
    borderline_rows, clear_rows = [], []
    for _, r in apps.iterrows():
        (borderline_rows if needs_llm(r) else clear_rows).append(r)

    print(f"[INFO] clear cases (no LLM): {len(clear_rows)}", flush=True)
    print(f"[INFO] borderline apps (LLM): {len(borderline_rows)}, batch size: {BATCH_SIZE_APPS}", flush=True)

    # Load cache; cache maps application_id -> LLMDecision dict
    cache = _load_cache(CACHE_PATH)
    print(f"[INFO] loaded cache entries: {len(cache)} from {CACHE_PATH}", flush=True)

    # Split borderline into "already cached" vs "needs call"
    to_call = []
    cached_rows = []
    for r in borderline_rows:
        aid = str(r[APP_ID_COL])
        if aid in cache:
            cached_rows.append(r)
        else:
            to_call.append(r)

    print(f"[INFO] borderline cached: {len(cached_rows)}  |  to call: {len(to_call)}", flush=True)

    results = []

    # 1) Process clear ones (no LLM call)
    if clear_rows:
        print(f"[INFO] processing clear cases...", flush=True)
    for idx, r in enumerate(clear_rows, start=1):
        base = baseline_rules_decision(r)
        llm = LLMDecision(
            decision=base,
            reasons=["Baseline strong enough: outside borderline region for DTI/FICO."],
            missing_documents=[],
            policy_citations=["UW-Policy-2.1 DTI Thresholds", "UW-Policy-3.4 FICO Bands"],
            risk_summary="Clear case per baseline thresholds; LLM not invoked.",
            fair_lending_flags=[]
        )
        combined = conservative_combine(base, llm)
        results.append({
            APP_ID_COL: str(r[APP_ID_COL]),
            "company_id": str(r["company_id"]),
            "baseline_decision": base,
            "llm_decision": llm.decision,
            "combined_decision": combined,
            "reasons": json.dumps(llm.reasons, ensure_ascii=False),
            "missing_documents": json.dumps(llm.missing_documents, ensure_ascii=False),
            "policy_citations": json.dumps(llm.policy_citations, ensure_ascii=False),
            "risk_summary": llm.risk_summary,
            "risk_score_0_1": llm.risk_score_0_1 if llm.risk_score_0_1 is not None else "",
            "fair_lending_flags": json.dumps(llm.fair_lending_flags, ensure_ascii=False),
            "notes": ""
        })
        if idx % 25 == 0 or idx == len(clear_rows):
            print(f"[INFO] clear processed: {idx}/{len(clear_rows)}", flush=True)

    # 2) Process cached borderline (fast)
    if cached_rows:
        print(f"[INFO] materializing {len(cached_rows)} cached borderline decisions...", flush=True)
    for idx, r in enumerate(cached_rows, start=1):
        aid = str(r[APP_ID_COL])
        base = baseline_rules_decision(r)
        d = cache.get(aid, {})
        # Validate cache payload defensively
        try:
            json_validate(instance=d, schema=LLM_DECISION_JSON_SCHEMA)
            llm = LLMDecision(**d)
        except Exception:
            llm = LLMDecision(
                decision=base,
                reasons=["Cache invalid; baseline applied."],
                missing_documents=[],
                policy_citations=["System-Fallback-1.0"],
                risk_summary="Cache invalid; baseline applied.",
                fair_lending_flags=[]
            )
        combined = conservative_combine(base, llm)
        results.append({
            APP_ID_COL: aid,
            "company_id": str(r["company_id"]),
            "baseline_decision": base,
            "llm_decision": llm.decision,
            "combined_decision": combined,
            "reasons": json.dumps(llm.reasons, ensure_ascii=False),
            "missing_documents": json.dumps(llm.missing_documents, ensure_ascii=False),
            "policy_citations": json.dumps(llm.policy_citations, ensure_ascii=False),
            "risk_summary": llm.risk_summary,
            "risk_score_0_1": llm.risk_score_0_1 if llm.risk_score_0_1 is not None else "",
            "fair_lending_flags": json.dumps(llm.fair_lending_flags, ensure_ascii=False),
            "notes": ""
        })
        if idx % 25 == 0 or idx == len(cached_rows):
            print(f"[INFO] cached processed: {idx}/{len(cached_rows)}", flush=True)

    # 3) Process remaining borderline in mini-batches (LLM), with time budget + incremental save
    start = time.time()
    total_call = len(to_call)
    if total_call:
        total_batches = math.ceil(total_call / BATCH_SIZE_APPS)
        print(f"[INFO] calling LLM for {total_call} borderline apps in {total_batches} batch(es)...", flush=True)

    for i in range(0, total_call, BATCH_SIZE_APPS):
        # Time budget check (between batches)
        elapsed = time.time() - start
        if elapsed > TIME_BUDGET_SEC:
            print(f"[WARN] time budget ({TIME_BUDGET_SEC}s) reached — stopping further LLM calls.", flush=True)
            break

        batch = to_call[i:i+BATCH_SIZE_APPS]
        batch_idx = (i // BATCH_SIZE_APPS) + 1
        print(f"[INFO] batch {batch_idx}/{total_batches}: {len(batch)} app(s)", flush=True)

        try:
            decisions_map = call_llm_batch_and_validate(batch, company_id)
        except Exception as e:
            print(f"[WARN] batch {batch_idx} failed ({e})", flush=True)
            if FALLBACK_TO_SINGLE:
                print(f"[INFO] falling back to single-item calls for this batch...", flush=True)
                decisions_map = {}
                for r in batch:
                    aid = str(r[APP_ID_COL])
                    try:
                        decisions_map[aid] = call_llm_and_validate(r, company_id)
                    except Exception as ee:
                        print(f"[WARN] single call failed for {aid} ({ee}); will use baseline.", flush=True)
            else:
                print(f"[INFO] falling back to baseline for entire batch.", flush=True)
                decisions_map = {}

        # append results; update cache; incremental save
        for r in batch:
            app_id = str(r[APP_ID_COL])
            base = baseline_rules_decision(r)
            llm = decisions_map.get(app_id)
            if llm:
                # store in cache
                cache[app_id] = llm.model_dump()
            else:
                llm = LLMDecision(
                    decision=base,
                    reasons=["LLM unavailable/failure for this app; baseline applied."],
                    missing_documents=[],
                    policy_citations=["System-Fallback-1.0"],
                    risk_summary="LLM call failed; baseline applied.",
                    fair_lending_flags=[]
                )
            combined = conservative_combine(base, llm)
            results.append({
                APP_ID_COL: app_id,
                "company_id": str(r["company_id"]),
                "baseline_decision": base,
                "llm_decision": llm.decision,
                "combined_decision": combined,
                "reasons": json.dumps(llm.reasons, ensure_ascii=False),
                "missing_documents": json.dumps(llm.missing_documents, ensure_ascii=False),
                "policy_citations": json.dumps(llm.policy_citations, ensure_ascii=False),
                "risk_summary": llm.risk_summary,
                "risk_score_0_1": llm.risk_score_0_1 if llm.risk_score_0_1 is not None else "",
                "fair_lending_flags": json.dumps(llm.fair_lending_flags, ensure_ascii=False),
                "notes": "" if llm.decision == base else "Conservative merge of baseline+LLM; review if disagreement."
            })

        # save cache + partial CSV after each batch
        _save_cache(CACHE_PATH, cache)
        partial_csv = os.path.join(OUTPUTS_DIR, f"loan_llm_decisions_{company_id}_partial.csv")
        pd.DataFrame(results).sort_values(APP_ID_COL).to_csv(partial_csv, index=False)
        print(f"[INFO] completed batch {batch_idx}/{total_batches}  |  partial saved → {partial_csv}", flush=True)

    # Final dataframe
    df = pd.DataFrame(results).sort_values(APP_ID_COL).reset_index(drop=True)
    print(f"[INFO] completed {len(df)} decision rows. Cache entries now: {len(cache)}", flush=True)
    return df

# Run with caching + time budget
decisions_df = run_batch_decisions(company_df, company_to_assess)
out_csv = os.path.join(OUTPUTS_DIR, f"loan_llm_decisions_{company_to_assess}.csv")
decisions_df.to_csv(out_csv, index=False)
print(f"[INFO] saved {len(decisions_df)} decisions -> {out_csv}", flush=True)

# Preview
decisions_df.head()


[INFO] preparing 5 applications for company COMP123...
[INFO] clear cases (no LLM): 1
[INFO] borderline apps (LLM): 4, batch size: 4
[INFO] loaded cache entries: 0 from /content/drive/Shareddrives/1a_Finance_LoanAI/loan/llm_cache_COMP123.json
[INFO] borderline cached: 0  |  to call: 4
[INFO] processing clear cases...
[INFO] clear processed: 1/1
[INFO] calling LLM for 4 borderline apps in 1 batch(es)...
[INFO] batch 1/1: 4 app(s)
[WARN] batch 1 failed (LLM batch failed after 1 attempts: LLM returned no data)
[INFO] falling back to single-item calls for this batch...
[WARN] single call failed for A100110 (LLM failed after 1 attempts: LLM returned no data); will use baseline.
[WARN] single call failed for A100211 (LLM failed after 1 attempts: LLM returned no data); will use baseline.
[WARN] single call failed for A100439 (LLM failed after 1 attempts: LLM returned no data); will use baseline.
[WARN] single call failed for A100473 (LLM failed after 1 attempts: LLM returned no data); will us

,application_id,company_id,baseline_decision,llm_decision,combined_decision,reasons,missing_documents,policy_citations,risk_summary,risk_score_0_1,fair_lending_flags,notes
0,A100110,COMP123,deny,deny,deny,"[""LLM unavailable/failure for this app; baseli...",[],"[""System-Fallback-1.0""]",LLM call failed; baseline applied.,,[],
1,A100211,COMP123,approve,approve,approve,"[""LLM unavailable/failure for this app; baseli...",[],"[""System-Fallback-1.0""]",LLM call failed; baseline applied.,,[],
2,A100439,COMP123,approve,approve,approve,"[""LLM unavailable/failure for this app; baseli...",[],"[""System-Fallback-1.0""]",LLM call failed; baseline applied.,,[],
3,A100450,COMP123,deny,deny,deny,"[""Baseline strong enough: outside borderline r...",[],"[""UW-Policy-2.1 DTI Thresholds"", ""UW-Policy-3....",Clear case per baseline thresholds; LLM not in...,,[],
4,A100473,COMP123,approve,approve,approve,"[""LLM unavailable/failure for this app; baseli...",[],"[""System-Fallback-1.0""]",LLM call failed; baseline applied.,,[],



---
## 8) Company-Level Narrative (Live LLM) — Matches Your Expected Structure

Produces a Markdown report like your example, referencing **lawsuit**, **debt-to-equity**, **credit score**, etc.  
It uses the **company profile** plus a small set of representative "chunks" that the model can cite.


In [ ]:
# ==== 8) Company-Level Narrative (Live LLM with HARD timeout + retries + fallback) ====

import json, time, os, random
from typing import Dict, Any

# Knobs (inherits CALL_TIMEOUT_SEC/BACKOFF_* from Step 2 / Step 6)
SUMMARY_TIMEOUT_SEC = float(os.getenv("SUMMARY_TIMEOUT_SEC", str(CALL_TIMEOUT_SEC)))  # per-call hard timeout
SUMMARY_RETRIES     = int(os.getenv("SUMMARY_RETRIES", "1"))  # extra attempts beyond the first
GENERATE_SUMMARY    = True  # set False to always use fallback narrative

def _make_company_chunks(profile: Dict[str, Any]) -> list[Dict[str, str]]:
    chunks = []
    if profile.get("lawsuit_flag"):
        chunks.append({"id": "Chunk 1", "text": profile.get("lawsuit_summary", "")})
    chunks.append({
        "id": "Financial Ratios",
        "text": (
            f"Debt-to-Equity: {profile['debt_to_equity']}; "
            f"Profit Margin: {profile['profit_margin']}; "
            f"ROA: {profile['roa']}; ROE: {profile['roe']}."
        )
    })
    chunks.append({"id": "Credit", "text": f"Experian credit score: {profile['credit_score']}."})
    return chunks

def _build_summary_prompt(profile: Dict[str, Any], decisions_df: pd.DataFrame) -> str:
    company_name = profile.get("company_name", profile["company_id"])
    chunks = _make_company_chunks(profile)
    sample = decisions_df[[APP_ID_COL, "combined_decision", "risk_summary"]].head(10).to_dict(orient="records")

    return f"""
System policy:
{LLM_SYSTEM_POLICY}

You are a senior credit underwriter. Create a concise Markdown report with the **exact** structure below.
Use the provided company profile metrics and context chunks. Keep the tone professional and specific.
If a lawsuit chunk exists, reference it as **Chunk 1** in Supporting Evidence.

Company: {company_name}
Company Key: {profile['company_id']}

Company Profile (JSON):
{json.dumps(profile, ensure_ascii=False, indent=2)}

Context Chunks (JSON):
{json.dumps(chunks, ensure_ascii=False, indent=2)}

Recent Sample Decisions (JSON, first 10 max):
{json.dumps(sample, ensure_ascii=False, indent=2)}

Please output **only** the following Markdown (no preface, no afterword):

Assessing loan risk for company: {profile['company_id']}...

--- Final Loan Risk Assessment and Recommendation ---
- **Key Risk Factors:**
    * **Lawsuit:** [If lawsuit_flag true, summarize the risk and uncertainty in 1–2 sentences. Otherwise omit this bullet.]
    * **Debt-to-Equity Ratio:** Explain what the value ({profile['debt_to_equity']}) means for leverage and resilience.
    * [Optionally include one more risk factor derived from ratios or credit score if warranted.]

- **Supporting Evidence:**

    * **Chunk 1:** [If a lawsuit chunk exists, cite its substance succinctly.]
    * **Financial Ratios:** Reflect Profit Margin ({profile['profit_margin']}), ROA ({profile['roa']}), ROE ({profile['roe']}).
    * **Credit Score:** Note the score ({profile['credit_score']}) and how it interacts with other risks.

- **Overall Risk Assessment:** [Low | Medium | High]

- **Loan Recommendation:** [Approve | Conditional / Further Review | Deny]

- **Justification:**

[Write 6–10 sentences covering strengths, risks, and specific next steps (e.g., legal review, scenario modeling, sensitivity analysis).]
"""

def _fallback_company_narrative(profile: Dict[str, Any], decisions_df: pd.DataFrame) -> str:
    # Deterministic, local-only narrative that mirrors your expected format
    company_key = profile["company_id"]
    lawsuit_bullet = ""
    evidence_chunk1 = ""
    if profile.get("lawsuit_flag"):
        lawsuit_bullet = (
            "    * **Lawsuit:** " +
            (profile.get("lawsuit_summary") or "Pending litigation; outcome and financial impact uncertain.") +
            "\n"
        )
        evidence_chunk1 = f"    * **Chunk 1:** {profile.get('lawsuit_summary', 'Pending litigation details.')}"

    # Simple risk heuristic for fallback
    d2e = float(profile["debt_to_equity"])
    credit = int(profile["credit_score"])
    risk_level = "Medium"
    if d2e >= 1.0 or credit < 680:
        risk_level = "High"
    elif d2e <= 0.4 and credit >= 760 and not profile.get("lawsuit_flag"):
        risk_level = "Low"

    # Recommendation heuristic
    recommendation = "Further Review" if profile.get("lawsuit_flag") else ("Approve" if risk_level == "Low" else "Conditional")

    return f"""Assessing loan risk for company: {company_key}...

--- Final Loan Risk Assessment and Recommendation ---
- **Key Risk Factors:**
{lawsuit_bullet if lawsuit_bullet else ""}    * **Debt-to-Equity Ratio:** The current ratio of {d2e} indicates leverage that affects resilience to shocks.
    * **Credit Profile:** Experian credit score of {credit} indicates {"strong" if credit>=740 else "moderate"} credit quality.

- **Supporting Evidence:**

{evidence_chunk1 if evidence_chunk1 else ""}    * **Financial Ratios:** Profit Margin {profile['profit_margin']}, ROA {profile['roa']}, ROE {profile['roe']}.
    * **Credit Score:** {credit}.

- **Overall Risk Assessment:** {risk_level}

- **Loan Recommendation:** {recommendation}

- **Justification:**

The company's operating metrics indicate {"solid" if profile['profit_margin']>=0.1 else "modest"} profitability with ROA {profile['roa']} and ROE {profile['roe']}. Debt-to-Equity at {d2e} affects flexibility and increases sensitivity to revenue or margin compression. The credit score of {credit} supports access to credit on reasonable terms{"; however, pending litigation adds uncertainty" if profile.get("lawsuit_flag") else ""}. We recommend {"a focused legal review and scenario analysis under adverse outcomes, including liquidity impacts and covenant headroom, before finalizing terms" if profile.get("lawsuit_flag") else "standard covenants and ongoing monitoring appropriate to the risk level"}. Sensitivity analysis on cash flows under rate shocks and downside revenue cases is advised to validate repayment capacity."""

def generate_company_narrative_safe(profile: Dict[str, Any], decisions_df: pd.DataFrame) -> str:
    if not GENERATE_SUMMARY:
        print("[INFO] summary generation disabled; using fallback.", flush=True)
        return _fallback_company_narrative(profile, decisions_df)

    prompt = _build_summary_prompt(profile, decisions_df)

    attempt = 0
    last_err = None
    while attempt <= SUMMARY_RETRIES:
        if THROTTLE_SECONDS:
            time.sleep(THROTTLE_SECONDS)
        try:
            print(f"[INFO] generating company narrative (attempt {attempt+1})...", flush=True)
            text = _call_with_hard_timeout(prompt, SUMMARY_TIMEOUT_SEC)  # from Step 6
            text = (text or "").strip()
            if not text:
                raise RuntimeError("Empty narrative from LLM.")
            print("[INFO] narrative generated via LLM.", flush=True)
            return text
        except Exception as e:
            last_err = e
            if attempt < SUMMARY_RETRIES:
                backoff = (BACKOFF_BASE_SEC ** (attempt+1)) + random.uniform(0, BACKOFF_JITTER)
                print(f"[WARN] narrative attempt {attempt+1} failed ({e}); retrying in {backoff:.2f}s...", flush=True)
                time.sleep(backoff)
                attempt += 1
            else:
                print(f"[WARN] narrative failed after {attempt+1} attempt(s): {e}. Using fallback.", flush=True)
                return _fallback_company_narrative(profile, decisions_df)

# ---- Run and save the narrative ----
print("[INFO] starting company-level narrative...", flush=True)
narrative_md = generate_company_narrative_safe(company_profile, decisions_df)
narrative_path = os.path.join(OUTPUTS_DIR, f"company_narrative_{company_to_assess}.md")
with open(narrative_path, "w", encoding="utf-8") as f:
    f.write(narrative_md or "(No narrative generated.)")
print(f"[INFO] saved company narrative → {narrative_path}", flush=True)
print(narrative_md[:1000] + ("..." if len(narrative_md) > 1000 else ""), flush=True)


[INFO] starting company-level narrative...
[INFO] generating company narrative (attempt 1)...
[WARN] narrative attempt 1 failed (LLM returned no data); retrying in 1.66s...
[INFO] generating company narrative (attempt 2)...
[WARN] narrative failed after 2 attempt(s): LLM returned no data. Using fallback.
[INFO] saved company narrative → /content/drive/Shareddrives/1a_Finance_LoanAI/loan/company_narrative_COMP123.md
Assessing loan risk for company: COMP123...

--- Final Loan Risk Assessment and Recommendation ---
- **Key Risk Factors:**
    * **Lawsuit:** Patent infringement lawsuit filed against Example Corp; potential damages material; outcome pending; impact assessment ongoing.
    * **Debt-to-Equity Ratio:** The current ratio of 0.8 indicates leverage that affects resilience to shocks.
    * **Credit Profile:** Experian credit score of 750 indicates strong credit quality.

- **Supporting Evidence:**

    * **Chunk 1:** Patent infringement lawsuit filed against Example Corp; potentia


---
## 9) Guardrails


In [ ]:

PROTECTED_EXAMPLE = {"race", "gender", "age", "religion", "marital_status"}
def assert_no_protected_columns(df: pd.DataFrame):
    bad = PROTECTED_EXAMPLE.intersection(set(map(str.lower, df.columns)))
    if bad:
        raise ValueError(f"Protected attributes detected in DataFrame: {sorted(bad)}. Remove before proceeding.")
assert_no_protected_columns(apps_df)
print("[TRACE] Protected-attr check passed. Total rows:", len(apps_df))


[TRACE] Protected-attr check passed. Total rows: 500



---
## 10) (Optional) Silence Tracing


In [ ]:

TRACE_LOGS = False
print("TRACE_LOGS set to", TRACE_LOGS)


TRACE_LOGS set to False
